In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import warnings
from sklearn import linear_model
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from math import sqrt
warnings.filterwarnings('ignore')

In [101]:
df = pd.read_csv('/Users/maralinetorres/Documents/GitHub/Predicting-Environmental-and-Social-Actions/Datasets/Environmental_impact_cleaned.csv')
df.head(3)

ISIN  Year       CompanyName         Country  \
0  DE0005545503  2016  1&1 DRILLISCH AG         Germany   
1  GB00B1YW4409  2010      3I GROUP PLC  United Kingdom   
2  GB00B1YW4409  2011      3I GROUP PLC  United Kingdom   

                                  Industry(Exiobase)  \
0                   Post and telecommunications (64)   
1  Financial intermediation, except insurance and...   
2  Financial intermediation, except insurance and...   

  EnvironmentalIntensity(Sales) EnvironmentalIntensity(OpInc)  \
0                        -0.07%                        -0.82%   
1                        -0.12%                        -0.11%   
2                        -0.16%                        -0.16%   

  TotalEnvironmentalCost WorkingCapacity FishProductionCapacity  ... SDG14.c  \
0               -539,318        -525,027                   -169  ...      -6   
1             -1,055,812      -1,032,103                   -277  ...      -4   
2               -961,875        -940,402                   -246  ...      -3   

  SDG15.1 SDG15.2 SDG15.5 %Imputed Env_intensity industry_avg  \
0      67      67     -22      23%       -0.0007    -0.020506   
1      51      51     -43      10%       -0.0012    -0.028537   
2      38      38     -39       9%       -0.0016    -0.028537   

  industry_avg_year Industry_indicator_year Environmental_Growth  
0         -0.020740                       1                  NaN  
1         -0.006402                       1                  NaN  
2         -0.009838                       1            33.333333  

[3 rows x 39 columns]

In [102]:
df = df.loc[:,['CompanyName','Year','Industry(Exiobase)','Env_intensity','Environmental_Growth']]
df.head()

CompanyName  Year                                 Industry(Exiobase)  \
0  1&1 DRILLISCH AG  2016                   Post and telecommunications (64)   
1      3I GROUP PLC  2010  Financial intermediation, except insurance and...   
2      3I GROUP PLC  2011  Financial intermediation, except insurance and...   
3      3I GROUP PLC  2012  Financial intermediation, except insurance and...   
4        3M COMPANY  2010  Activities of membership organisation n.e.c. (91)   

   Env_intensity  Environmental_Growth  
0        -0.0007                   NaN  
1        -0.0012                   NaN  
2        -0.0016             33.333333  
3        -0.0015             -6.250000  
4        -0.0790                   NaN


## Industries Regressions 


filter industries more than 3 companies

In [103]:
df_industry = df.groupby('Industry(Exiobase)').count()['CompanyName'].reset_index()
industries = df_industry[df_industry['CompanyName'] > 3]['Industry(Exiobase)']
df_industry_count4 = df[df['Industry(Exiobase)'].isin(industries)]
df_industry_count4.head()

In [106]:
## I recommend to run the regression by itself for this industry and compare the results with the function
data = df_c[df_c['Industry(Exiobase)'] == 'Activities auxiliary to financial intermediation (67)']
data.shape

(93, 5)

In [131]:
years = [2016, 2017, 2018]
outcomeYear = 2019
df_c = df_industry_count4.copy()
def calculateIndustriesRegressions(outcomeYear, pastYears, df_c):
    years.sort()
    industry_regressions = {}
    for i in np.unique(df_c['Industry(Exiobase)']):
        for year in years:
            data = df_c[(df_c['Industry(Exiobase)'] == i) & (df_c['Year'] == year)]
            data = data.loc[:,['CompanyName','Env_intensity']]
            data.rename(columns={'Env_intensity': f'Env_intensity_{year}'}, inplace=True) 
            if(year == min(years)):
                data1 = pd.DataFrame(data)
            else:
                data2 = pd.merge(data1, data, on=["CompanyName"])
                data1 = data2.copy()

        data3 = df_c[df_c['Year'] == outcomeYear]
        data3 = data3[['CompanyName','Env_intensity']]
        data3.rename(columns={'Env_intensity': f'Env_intensity_{outcomeYear}'}, inplace=True) 
        data3 = pd.merge(data3, data2, on=["CompanyName"])

        filter_col = [col for col in data3 if ((col.startswith('Env_intensity') and not(col.endswith(f'{outcomeYear}'))))]
        outcome_col = [col for col in data3 if (col.startswith('Env_intensity') and col.endswith(f'{outcomeYear}'))]
        X=data3[filter_col]
        y=data3[outcome_col]

        if(X.shape[0] > 0):
            regr = linear_model.LinearRegression()
            X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
            regr.fit(X_train, y_train)
            y_train_pred = regr.predict(X_train)
            y_pred = regr.predict(X_test)

        industry_regressions[i] = {'OutcomeYear':outcomeYear, 'MSE_test': metrics.mean_squared_error(y_test, y_pred),'Intercept': regr.intercept_,'Coefficient': regr.coef_, 
                                   'R2_score':metrics.r2_score(y_test, y_pred)}
        
    return industry_regressions

In [132]:
calculateIndustriesRegressions(2019, years, df_c)

{'Activities auxiliary to financial intermediation (67)': {'OutcomeYear': 2019,
  'MSE_test': 0.0016655323522020633,
  'Intercept': array([0.0367213]),
  'Coefficient': array([[ 17.97654644, -36.23640739,  17.3996719 ]]),
  'R2_score': -368.5707882105169},
 'Activities of membership organisation n.e.c. (91)': {'OutcomeYear': 2019,
  'MSE_test': 0.00020973148014216605,
  'Intercept': array([0.02800866]),
  'Coefficient': array([[-0.13309253,  0.94096813,  0.53247183]]),
  'R2_score': 0.9818286119890922},
 'Air transport (62)': {'OutcomeYear': 2019,
  'MSE_test': 0.07989444548137743,
  'Intercept': array([0.00012204]),
  'Coefficient': array([[ 0.36741789, -0.56379912,  1.19202933]]),
  'R2_score': 0.24313719294477687},
 'Chemicals nec': {'OutcomeYear': 2019,
  'MSE_test': 0.005144820159546412,
  'Intercept': array([0.00515047]),
  'Coefficient': array([[0.37239771, 0.43783879, 0.17308092]]),
  'R2_score': 0.7934523826330316},
 'Collection, purification and distribution of water (41)': {